# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status - семейное положение
- family_status_id - идентификатор семейного положения
- gender - пол клиента
- income_type - тип занятости
- debt - имел ли задолженности по возварту кредита
- total_income - ежемесячный доход
- purpose - цель получения кредита

### Шаг 1. Открываю файл с данными и изучаю общую информацию. 

#### Подключаю нужные библиотеки

In [2]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')

Installing mystem to /Users/mariagogoleva/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


#### Открываю таблицу и изучаю общую информацию о данных

In [2]:
data_customers = pd.read_csv('data.csv')
print(data_customers.info())
data_customers.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Данные представлены в виде 12 столбцов. Всего 21525 записей. 
В столбцах days_employed и total_income есть пропущенные значения. 
2 столбца имеют формат вещественных чисел, 5 столбцов - целочисленные и еще 5 столбцов содержат категориальные переменные. 
Пропуски данных скорее всего являются случайными.

### Шаг 2. Предобработка данных

### Обработка пропусков

#### Определяю и заполняю пропущенные значения

In [3]:
print(data_customers[data_customers['days_employed'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


Получается, что в столбцах days_employed и total_income 2174 пропущенных значения.

In [4]:
data_customers['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Заполняю пропуски в соответствии с типом дохода. 

In [5]:
total_income_medians = data_customers.groupby('income_type')['total_income'].median().to_dict()
days_employed_medians = data_customers.groupby('income_type')['days_employed'].median().to_dict()

def replace_na(row, median_dict, col):
    if pd.isnull(row[col]):
        return median_dict[row['income_type']]
    else:
        return row[col]
    
for col, _dict in zip(['total_income','days_employed'],
                      [total_income_medians,days_employed_medians]):
    data_customers[col] = data_customers.apply(lambda row: replace_na(row,_dict,col),axis=1)

In [6]:
data_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из возможной причины пропусков - неверно собраны данные о клиентах.
Пропуски заполнены по принципу замены отсутствующих значений на медианные в соответствии с типом дохода.

### Вывод

В 2174 строках пропущены значения в столбцах days_employed и total_income. 
Пропущенные значения заменены на медианные для каждого из столбцов.
Таким образом сохранены данные.

#### Проверю данные о количестве детей в семье.

In [7]:
print(data_customers['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Из этих данных видно, что данные о кличестве детей -1 и 20 некорректны. Скорее всего знаки минус и 0 поставлены случайно. 

In [8]:
data_customers.loc[data_customers['children'] == -1, 'children'] = 1
data_customers.loc[data_customers['children'] == 20, 'children'] = 2

In [9]:
print(data_customers['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Заменила некорректные данные. Выведены обновленная информация по столбцу "Дети".

Здесь лучше использовать заполнение пропусков в соответствии с типом дохода. Тогда пенсионеры будут получать также, как и другие пенсионеры и т.д. Все-таки в данных есть достаточное число групп, заполнять пропуски в стаже, скажем, у студентов и пенсионеров одним и тем же значением не очень то правильно. То же самое касается и столбца с доходами: пропуски у предпринимателя и пенсионера заполняются одним числом, что вызывает вопросы. 

### Замена типа данных

Заменяю вещественный тип данных на целочисленный. 
Метод выбирала с учетом того,что нужно заменить тип данных в столбце именно на целочисленный, поэтому не применяла
to_numeric

In [10]:
data_customers['days_employed'] = data_customers['days_employed'].astype('int')
data_customers['total_income'] = data_customers['total_income'].astype('int')

### Вывод

В столбцах days_employed и total_income, debt изменен формат данных на целочисленный. 

### Обработка дубликатов

Удаляю дубликаты в данных. Меняю регистр букв в столбце "Образование" на строчные. 
Таким образом все данные в этом столбце приведены к единому виду.

In [11]:
data_customers['education'] = data_customers['education'].str.lower()

Удаляю дубликаты с сохранением индексов. 
Из возможной причины появления дубликатов - неверно собраны данные о клиентах, 
копирование баз данных из разных источников.

In [12]:
data_customers = data_customers.drop_duplicates().reset_index(drop=True)
data_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Из данных удалены строки с пропущенными значениями. 

Использован метод drop_duplicates() использован. Из данных удалены именно дубликаты, а не пропусщенные значения.

### Лемматизация

In [13]:
purpose_unique = data_customers['purpose'].unique()
purpose_unique

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Выделяю леммы в значениях столбца с целями получения кредита.
Создаю словаль с 5 видами целей и в теле функции проверяю совпадает ли каждое значение в столбце со словарем.
При совпадении заменяю текущее значение в столбце на слово из словаря.

In [14]:
def grouped_purposes(purpose):
    purpose_dict = ['свадьба', 'недвижимость', 'жилье', 'автомобиль', 'образование']
    return list(set(m.lemmatize(purpose)) & set(purpose_dict))[0]

data_customers['purpose'] = data_customers['purpose'].apply(grouped_purposes)

### Вывод

Были выделены 5 основных целей взятия кредита. 
Все остальные формулировки были заменены на одну из: 'свадьба', 'недвижимость', 'жилье', 'автомобиль', 'образование'.

### Категоризация данных

#### Выявлю сколько раз встречается каждый тип(цель) обращений.

In [15]:
print(data_customers['purpose'].value_counts())

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose, dtype: int64


### Вывод

Посчитала количество записей для каждой цели получения кредита. Больше всего кредитов берут для покупки недвижимости, а меньше всего людей обращаются за кредитом с целью организации свадьбы. 

#### Проведу анализ уровня доходов клиентов.

In [16]:
print(data_customers['total_income'].describe())

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64


In [17]:
max_income = data_customers['total_income'].max()
min_income = data_customers['total_income'].min()
median_income = data_customers['total_income'].median()
mean_income = data_customers['total_income'].mean()
print('Максимальный ежемесячный доход', max_income, 'руб.')
print('Минимальный ежемесячный доход', min_income, 'руб.')
print('Медианный ежемесячный доход', median_income, 'руб.')
print('Средний ежемесячный доход', mean_income, 'руб.')

Максимальный ежемесячный доход 2265604 руб.
Минимальный ежемесячный доход 20667 руб.
Медианный ежемесячный доход 142594.0 руб.
Средний ежемесячный доход 165319.57229421087 руб.


### Шаг 3. Исследование данных

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
children_debt = data_customers.groupby('children')['debt'].sum()
print(children_debt)

children
0    1063
1     445
2     202
3      27
4       4
5       0
Name: debt, dtype: int64


### Вывод

Есть зависимость между наличием детей и возвратом кредита в срок.
Как ни странно, но задолженностей по кредитам больше у заемщиков без детей.
В целом - чем меньше детей в семье, тем больше задолженностей имеют клиенты по возрату кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
data_pivot = data_customers.pivot_table(index=['family_status'], columns='purpose', values='debt', aggfunc='sum').fillna(value = 0)
data_pivot['Количество задолженностей по возврату кредита'] = data_pivot['автомобиль'] + data_pivot['жилье'] + data_pivot['недвижимость'] + data_pivot['образование'] + data_pivot['свадьба']
data_pivot

purpose,автомобиль,жилье,недвижимость,образование,свадьба,Количество задолженностей по возврату кредита
family_status,,,,,,
Не женат / не замужем,82.0,49.0,81.0,62.0,0.0,274.0
в разводе,21.0,19.0,28.0,17.0,0.0,85.0
вдовец / вдова,20.0,12.0,16.0,15.0,0.0,63.0
гражданский брак,51.0,36.0,55.0,60.0,186.0,388.0
женат / замужем,229.0,192.0,294.0,216.0,0.0,931.0


In [20]:
status_debt = data_customers.groupby('family_status')['debt'].sum()
print(status_debt)

family_status
Не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
Name: debt, dtype: int64


### Вывод

Есть зависимость между семейным положением и возвратом кредита в срок. 
Наибольшее количество задолженностей по кредитам имеется у клиентов, которые женаты/замужем, либо состоят в гражданском браке.
Свободные люди гораздо меньше имеют просрочек по платежам, нежели семейные люди.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
small_income_sum = data_customers.loc[data_customers.loc[:, 'total_income'] <= 1.425940e+05]['debt'].sum()
high_income_sum = data_customers.loc[data_customers.loc[:, 'total_income'] > 1.425940e+05]['debt'].sum()


print('Количество прострочек по платежам среди людей с небольшим доходом:', small_income_sum)
print('Количество прострочек по платежам среди людей с большим доходом:', high_income_sum)

Количество прострочек по платежам среди людей с небольшим доходом: 910
Количество прострочек по платежам среди людей с большим доходом: 831


### Вывод

Для выявления зависимости между уровнем дохода и возвартом кредита в срок, суммирую количество задолженностей по платежам в зависимости от уровня дохода, больше медианы и меньше медианы.
Количество просрочек у людей с большим доходом меньше, хотя эти величины и не сильно различаются.
Я бы сказала, что уровень дохода - не критичный параметр для оценки платежеспособности клиента при выдаче кредита.

- Как разные цели кредита влияют на его возврат в срок?

In [22]:
purpose_debt = data_customers.groupby('purpose')['debt'].sum()
print(purpose_debt)

purpose
автомобиль      403
жилье           308
недвижимость    474
образование     370
свадьба         186
Name: debt, dtype: int64


### Вывод

Цели кредита не имеют сильной взаимосвязи с возвратом кредита в срок. Можно отметить, что наименьшее количество задолженностей имеют клиенты, которые брали кредит, чтобы сыграть свадьбу.

### Шаг 4. Общий вывод

В этом проекте нужно было разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Входные данные были представлены в виде 12 столбцов. Всего 21525 записей. 
Была проведена предобработка данных: обработка пропусков, замена некорректных значений, обработка дубликатов, замена типа данных, категоризация и лемматизация данных. 

Далее была проанализирована взаимосвязь между различными параметрами платежеспособности клиента.

Так, уровень дохода - не критичный параметр для оценки платежеспособности клиента при выдаче кредита.
Свободные люди гораздо меньше имеют просрочек по платежам, нежели семейные люди.
Как ни странно, но задолженностей по кредитам больше у заемщиков без детей.